### Critic data creation

In [ ]:
import json
with open("../output/testtime1_1105_dev.json", 'r') as f:
    dev_data = json.load(f)
len(dev_data)

500

In [13]:
real_count = 0
false_count = 0
for d in dev_data:
    content = d['instruction'] + d['input']
    completion = client.chat.completions.create(
    model="/media/disk1/chatgpt/LLaMA-Factory/saves/llama3-8b/full/sft/", 
    messages=[
        {"role": "user", "content": content}
    ]
    )
    if completion.choices[0].message.content == d['output']:
        real_count += 1
    else:
        print(completion.choices[0].message.content, d['output'])
        false_count += 1

 [No Retrieval]
 [No Retrieval]


In [4]:
model.invoke('你好')

AIMessage(content='你好！有什么我可以帮助你的吗？', additional_kwargs={'refusal': None}, response_metadata={'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-ab72e491-733a-43cc-881f-61f90729e518-0', usage_metadata={'input_tokens': 8, 'output_tokens': 28, 'total_tokens': 36, 'input_token_details': {}, 'output_token_details': {}})

In [ ]:
import json
with open("../data/WebQSP.json", "r") as f:
    data = json.load(f)
print(len(data))

In [2]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.chains import LLMChain


multi_graph_three_way = """You will be provided with a query, evidence, output sentence, and preceding sentences (optional). Your task is to determine whether the information in the output sentence can be fully verified by the evidence or if it requires further external verification. There are three cases:\n\n- If the output sentence can be verified solely with the evidence and the preceding sentences, then respond with [No Retrieval].\n- If additional information about the tail entity in evidence is needed to verify the output sentence, respond with [Continue to Retrieve Evidence]\n- If more information unrelated to the evidence is needed, e.g. totally new relationship or new entity, reponse with [New Retrieval]. Please provide explanations for your judgments. \n\n\n###\nQuery: {query}\n\nPreceding sentences: {preceding_sentences}\n\nEvidende: {evidence}
Output: {target_output}
Rating: \n"""
multi_graph_relevance =  """You'll be provided with a query, along with evidence and possibly some preceding sentences. \nYour job is to determine if the evidence is relevant to the initial query and the preceding context. \nIf the evidence meets this requirement, respond with [Relevant]; otherwise, generate [Irrelevant].\n\n\n###\nQuery: {query}\n\nPreceding sentences: {preceding_sentences}\n\nEvidende: {evidence}
Rating: """

multi_graph_groundness =  """You'll be provided with a query, along with evidence and possibly some preceding sentences. \nYour task is to evaluate if the query can be fully supported by the information provided in the evidence, and provide explanations on your judgement.\nUse the following entailment scale to generate a score:\n[Fully supported] - All information needed is supported by the evidence. \n[Partially supported] - The query is supported by the evidence to some extent, but there is major information that is not discussed in the evidence. For example, if the query asks about two concepts and the evidence only discusses either of them, it should be considered a [Partially supported].\n[No support / Contradictory] - The query is completely unrelated to the evidence, or contradicts the evidence.\n###\nQuery: {query}\n\nPreceding sentences: {preceding_sentences}\n\nEvidende: {evidence}
Rating: \n"""

multi_graph_utility = """\nGiven a query and an output, rate whether the response appears to be a helpful and informative answer to the query, from 1 (lowest) - 5 (highest). We call this score perceived utility.\n\nThe detailed criterion is as follows:\n5: The response provides a complete, highly detailed, and informative response to the query, fully satisfying the information needs.\n4: The response mostly fulfills the need in the query, while there can be some minor improvements such as discussing more detailed information, having better structure of the response, or improving coherence.\n3: The response is acceptable, but some major additions or improvements are needed to satisfy users' needs.\n2: The response still addresses the main request, but it is not complete or not relevant to the query.\n1: The response is barely on-topic or completely irrelevant.\n##\n\nQuery: Who is the current prime minister of the UK as of 2023?\nOutput: Boris Johnson was the prime minister of the UK from 2019 - 2022.\nPerceived utility: 2\nExplanation: While the output provides a factually correct statement about the UK prime minister from 2019 to 2022, this instruction asks who the prime minister is as of 2023, so it doesn't answer the instruction. Therefore, the utility is 2.\n\n##\n\nQuery: Given a description of a travel destination, recommend 10 tourist attractions with detailed explanations of each. The travel destination is Tokyo, Japan.\nOutput: 'Tokyo is a vibrant city full of exciting tourist attractions. Some must-see sights include the Tokyo Skytree, Tokyo Disneyland, Sensoji Temple, Meiji Shrine, Tsukiji Fish Market, Harajuku, and Shinjuku Gyoen.\nPerceived utility: 3\nExplanation: This output doesn't provide descriptions of each attraction and the number of the attractions is also less than 10. While this output partially answers the instructions, it doesn't match the instructions strictly.\n\n##\n\nQuery: {query}\nOutput:{output}\n"""

multi_graph_three_way_prompt = PromptTemplate(input_variables=["query", "preceding_sentences", "evidence", "target_output"], template=
multi_graph_three_way)
multi_graph_relevance_prmpt = PromptTemplate(input_variables=["query", "preceding_sentences", "evidence"], template=
multi_graph_relevance)
multi_graph_groundness_prompt = PromptTemplate(input_variables=["query", "preceding_sentences", "evidence",], template=
multi_graph_groundness)
multi_graph_utility_prompt = PromptTemplate(input_variables=["query", "output"], template=multi_graph_utility)
# prompt_template = FewShotPromptTemplate(\n#         examples=examples_dict,\n#         example_prompt=examples_prompt,\n#         prefix=\n#         """\n# Based on the Table below, your task is to accurately output columns related to the query or contain useful information about the query. This process involves linking similar words or semantically similar terms to columns in the Table.\n# Approach this task as follows:\n# Read the query and extra information thoroughly and list every possible link from query term to column in the Table. \n# Then based on the column linking, output all useful columns at last. Make sure all columns in the linking step are included and every column is in the Table.""",\n#         suffix=\n#         """\n# Table: {table}\n# Extra information: {aug}\n\n# Query: {claim}""",\n#         input_variables=["table", "claim", "aug"],\n# )\n# "[Continue to Retrieve Evidence]\n\nExplanation: The evidence provided only mentions Viggo Mortensen as a film actor, but does not provide any information about his role as Aragorn. To verify if Aragorn influenced Samuel Taylor Coleridge, we would need additional information about Viggo Mortensen's portrayal of Aragorn in a specific context that could have influenced Coleridge."


In [ ]:
llm_chain = LLMChain(llm=model, prompt=multi_graph_relevance_prmpt, verbose=True)
batch_pred = llm_chain.batch([{"query": data[0]['RawQuestion'], "preceding_sentences": "", "evidence": " relationship: influence.influence_node.influenced_by",}], return_only_outputs=True)


In [ ]:
batch_pred[0]['text']

In [ ]:
llm_chain = LLMChain(llm=model, prompt=examples_prompt, verbose=True)
batch_pred = llm_chain.batch([{"query": data[0]['RawQuestion'], "preceding_sentences": "", "evidence": "(Viggo Mortensen, film.actor.film, Unknown_Name)", "target_output": "Aragorn"}], return_only_outputs=True)


In [31]:
PROMPT_DICT = {
    "ground_instruction": (
        "You will be given a query and evidence. Your objective is to assess the extent to which the query is supported by the information presented in the evidence.\n"
        "Rate the level of support from [Ignore / Contradictory], [Little support], [Partially supported], [Mostly supported], [Fully supported]."
    ),
    "ground_input": (
        "Query: {query}\n"
        "Evidence: {evidence}"
    ),
    "ground_instruction_multi": (
        "You will receive a query, evidence, and output, and optional preceding sentences. Your task is to evaluate whether the output filters out valid information from the evidence to answer the given query, and provide explanations on your judgement\n"
        "Use the following entailment scale to generate a score:\n"
        "[Fully supported] - the output contains sufficient information needed to answer the query, fully satisfying the information needs.\n"
        "[Partially supported] - The output contains information to some extent, but there is major information in evidence that is not discussed in the output. For example, if a query asks about two concepts and the output only discusses either of them, it should be considered a [Partially supported].\n" 
        "[No support / Contradictory] - The output completely ignores evidence, or is unrelated to the query.\n\n"
        # "Make sure to not use any external information/knowledge to judge whether the output is true or not.\n\n"
    ),
    "relevance_instruction": (
        "You will receive a query, topic entity, evidence and optional preceding sentences containing history information. The evidence contains graph relationships or entities may be useful to answer the query. Your task is to filters out 3 valid information from the evidence that contribute to answering the query and provide a relevance score for each output.\n"
        "The score of relevance range from [Fully Relavant], [Partially Relevant] to [Unrelevant]."
    ),
    "relevance_input": (
        "Query: {query}\n"
        "Topic entity: {topic}\n"
        "Evidence: {evidence}\n"
        "Preceding sentences: {preceding_sentences}"
    ),
    "entity_instruction": (
        "You will receive a query, evidence and optional preceding sentences containing history information. The evidence contains the associated retrieved knowledge graph triplets (head_entity, relation, tail_entity). Your task is to rate relevance score for each tail_entity from the evidence."
        "The score of relevance range from [Fully Relevant], [Partially Relevant] to [Unrelevant]."
    ),
    "entity_input": (
        "Query: {query}\n"
        "Evidence: {evidence}\n"
        "Preceding sentences: {preceding_sentences}"
    ),
    "ground_multi_input": (
        "Task instruction: {instruction}\n"
        "Preceding sentences: {preceding_sentences}\n"
        "Output: {target_output}\n"
        "Evidence: {evidence}"
    ),
    "ground_multi_input_wo_preceding": (
        "Task instruction: {instruction}\n"
        "Output: {target_output}\n"
        "Evidence: {evidence}"
    ),
    "retrieval_multi_instruction": (
        "You will be provided with an instruction, evidence, output sentence, and preceding sentences (optional). If the preceding sentence is given, the output should be the sentence that follows those preceding sentences.  Your task is to determine whether the information in the output sentence can be fully verified by the evidence or if it requires further external verification. If the output sentence can be verified solely with the evidence or doesn’t require any verification, respond with [No Retrieval]. If additional information is needed to verify the output sentence, respond with [Retrieval]. Please provide explanations for your judgments.\n\n" 
    ),
    "retrieval_multi_input": (
        "Task instruction: {instruction}\n"
        "Preceding sentences: {preceding_sentences}\n"
        "Evidence: {evidence}\n"
        "Output: {target_output}"
    ),
    "multi_retrieval_three_way_instruction": (
        "You will be provided with a query, evidence, output sentence, and preceding sentences (optional). Your task is to determine whether the information in the output sentence can be fully verified by the evidence or if it requires further external verification. There are three cases:\n" 
        "- If the output sentence can be verified solely with the evidence and the preceding sentences, then respond with [No Retrieval]. \n"
        "- If additional information about the tail entity in evidence is needed to verify the output sentence, respond with [Continue to Retrieve Evidence] \n"
        "- If more information unrelated to the evidence is needed, e.g. totally new relationship or new entity, reponse with [New Retrieval].\n\n"
    ),
    "multi_retrieval_three_way_input": (
        "Query: {query}\n"
        "Preceding sentences: {preceding_sentences}\n"
        "Evidence: {evidence}\n"
        "Output: {target_output}"
    ),
     "multi_retrieval_three_way_input_wo_preceding": (
        "Query: {query}\n"
        "Evidence: {evidence}\n"
        "Output: {target_output}"
    ),
    "utility_instruction": (
        "Given an instruction and an output, rate whether the response appears to be a helpful and informative answer to the query, from 1 (lowest) - 5 (highest). We call this score perceived utility.\n"
        "[Utility:5]: The response provides a complete, highly detailed, and informative response to the query, fully satisfying the information needs.\n"
        "[Utility:4]: The response mostly fulfills the need in the query, while there can be some minor improvements such as discussing more detailed information, having better structure of the response, or improving coherence. \n"
        "[Utility:3]: The response is acceptable, but some major additions or improvements are needed to satisfy users' needs.\n"
        "[Utility:2]: The response still addresses the main request, but it is not complete or not relevant to the query.\n"
        "[Utility:1]: The response is barely on-topic or completely irrelevant.\n"
    ),
    "utility_input": (
        "Task instruction: {instruction}\n"
        "Output: {output}"
    ),
}

In [ ]:
import json
#解决retrieve entity 过多的问题
with open("../data/WebQSP.json", "r") as f:
    data = json.load(f)
input_data = []
for line in data:
    candidate_answer = []
    relation_paths = []
    continue_paths = []
    for p in line['Parses']:
        candidate_answer.extend([ans["EntityName"] for ans in p['Answers'] if ans["EntityName"] and ans["EntityName"] not in candidate_answer ])
        if p['InferentialChain'] and p['InferentialChain'] not in relation_paths:
            relation_paths.append(p['InferentialChain'])
    for path in relation_paths:
        if len(path) > 1:
            continue_paths.append([path[:i] for i in range(len(path))])
    input_data.append({"input": {"query": line['RawQuestion'], "preceding_sentences": "", "evidence": "", "target_output": ';'.join(candidate_answer)}, "decision_token": "[New Retrieval]"})
    for c in continue_paths:
        if len(c) == 1:
            input_data.append({"input": {"query": line['RawQuestion'], "preceding_sentences": "", "evidence": f"Relations retrieved: {c[-1]}", "target_output": ';'.join(candidate_answer)}, "decision_token": "[Continue to Retrieve Evidence]"})
        else:
            input_data.append({"input": {"query": line['RawQuestion'], "preceding_sentences": f"Relations retrieved: {c[:-1]}", "evidence": f"Relations retrieved: {c[-1]}", "target_output": ';'.join(candidate_answer)}, "decision_token": "[Continue to Retrieve Evidence]"})
    #生成relation path，根据relation path构造
    for p in relation_paths:
        if len(p) ==1:
            input_data.append({"input": {"query": line['RawQuestion'], "preceding_sentences": "", "evidence": f"{p[-1]}", "target_output": ';'.join(candidate_answer)}, "decision_token": "[No Retrieval]"})
        else:
            input_data.append({"input": {"query": line['RawQuestion'], "preceding_sentences": f"Relations retrieved: {p[:-1]}", "evidence": f"Relations retrieved: {p[-1]}", "target_output": ';'.join(candidate_answer)}, "decision_token": "[No Retrieval]"})



In [1]:
from collections import Counter
def create_retrieval_data(input_data, multi_retrieval=False):
    print("creating multi sentence retrieval data")
    processed_data = []
    for item in input_data:
        input = item["input"]
        output = item["decision_token"]
        if len(str(output)) == 0:
            continue
        if output not in [ "[New Retrieval]", "[Continue to Retrieve Evidence]", "[No Retrieval]"]:
            continue

        if len(input["preceding_sentences"]) == 0:
            processed_data.append({"instruction": PROMPT_DICT["multi_retrieval_three_way_instruction"], "input": PROMPT_DICT["multi_retrieval_three_way_input_wo_preceding"].format_map(input), "output": output, "task": "multi_retrieval"})
        else:
            processed_data.append({"instruction": PROMPT_DICT["multi_retrieval_three_way_instruction"], "input": PROMPT_DICT["multi_retrieval_three_way_input"].format_map(input), "output": output, "task": "retrieval"})
            
    print(processed_data[-1])
    print("total data number: {}".format(len(processed_data)))
    print(Counter([item["output"] for item in processed_data ]))
    return processed_data


In [ ]:
import random
import json
from collections import Counter
def create_relevance_data(input_data):
    print("creating relevance data")
    processed_data = []
    for item in input_data:
        output = ''
        match_dict= item["match"]
        for r, s in match_dict.items():
            output += f"{r}{s}; "
        # if label == "[Relevant]" and random.random() > 0.7:
        #     continue
        processed_data.append({"instruction": PROMPT_DICT["relevance_instruction"], "input": PROMPT_DICT["relevance_input"].format_map(item['input']), "output": output, "task": "relation_relevance"})
    print(processed_data[-1])
    print("total data number: {}".format(len(processed_data)))
    # print(Counter([item["output"] for item in processed_data ]))
    return processed_data

def create_entity_data(input_data):
    print("creating entity data")
    processed_data = []
    
    for item in input_data:
        output = ''
        match_dict= item["match"]
        for r, s in match_dict.items():
            output += f"{r}{s}; "
        # if label == "[Relevant]" and random.random() > 0.7:
        #     continue
        processed_data.append({"instruction": PROMPT_DICT["entity_instruction"], "input": PROMPT_DICT["entity_input"].format_map({"query": item['query'], "evidence": item['evidence'], "preceding_sentences": ""}), "output": output, "task": "entity_relevance"})
    print(processed_data[-1])
    print("total data number: {}".format(len(processed_data)))
    # print(Counter([item["output"] for item in processed_data ]))
    return processed_data


In [56]:
with open('../training_data/entity_relation_relevance.json', 'w') as f:
    json.dump(relevance_data + entity_data, f)

In [52]:
entity_data = create_entity_data(relevance_entity)

creating entity data
{'instruction': 'You will receive a query, evidence and optional preceding sentences containing history information. The evidence contains the associated retrieved knowledge graph triplets (head_entity, relation, tail_entity). Your task is to rate relevance score for each tail_entity from the evidence.The score of relevance range from [Fully Relavant], [Partially Relevant] to [Unrelevant].', 'input': 'Query: what do you call russian currency\nEvidence: (Russia, location.country.currency_used, Russian ruble)Preceding sentences: ', 'output': 'Russian ruble[Fully Relavant]; ', 'task': 'relevance'}
total data number: 6110


In [3]:
def create_groundness_data(input_data, multi_retrieval=False):
    print("creating groundness data")
    processed_data = []
    for item in input_data:
        if multi_retrieval is True:
            if "sent_idx" not in item or item["sent_idx"] == 0 or len(item["preceding_sentences"]) == 0:
                processed_data.append({"instruction": PROMPT_DICT["ground_multi_instruction"], "input": PROMPT_DICT["ground_multi_input_wo_preceding"].format_map(input), "output": item["score"], "task": "groudness"})
            else:
                processed_data.append({"instruction": PROMPT_DICT["ground_multi_instruction"], "input": PROMPT_DICT["ground_multi_input"].format_map(input), "output": item["score"], "task": "groudness"})
        else: 
            processed_data.append({"instruction": PROMPT_DICT["ground_instruction"], "input": PROMPT_DICT["ground_input"].format_map(item['input']), "output": item["score"], "task": "groudness"})
    print(processed_data[-1])
    print("total data number: {}".format(len(processed_data)))
    print(Counter([item["output"] for item in processed_data ]))
    return processed_data


#### complete chain create

In [5]:
import os
from langchain_openai import AzureOpenAIEmbeddings

os.environ["AZURE_OPENAI_API_KEY"] = "2b219db0d2984f9dae28b651ab8ab3d9"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://smsh.openai.azure.com/"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-03-01-preview"
embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-3-small",
)

from langchain.storage import LocalFileStore, RedisStore
from langchain.embeddings import CacheBackedEmbeddings
from langchain_community.vectorstores import FAISS
# store = RedisStore(redis_url="redis://localhost:6379")
# cached_embedder = CacheBackedEmbeddings.from_bytes_store(
# embeddings, store, namespace="openai"
# )
# row_string = []
# with open('../data/clean_relations', 'r') as f:
#     data = f.readlines()
# db = FAISS.from_texts(data, cached_embedder)
# retriever = db.as_retriever(search_kwargs={"k": 5})

In [2]:
import sys
sys.path.append('/media/disk1/chatgpt/zh/graph_data')
from src.graph_utils import *

In [5]:
import datasets
import networkx as nx
from collections import deque
import walker
import json
def build_Digraph(graph: list) -> nx.Graph:
    G = nx.DiGraph()
    for triplet in graph:
        h, r, t = triplet
        G.add_edge(h, t, relation=r.strip())
    return G

def bfs_with_rule(graph, start_node, target_rule, max_p = 10):
    result_paths = []
    queue = deque([(start_node, [])])
    while queue:
        current_node, current_path = queue.popleft()
        if len(current_path) == len(target_rule):
            result_paths.append(current_path)
        if len(current_path) < len(target_rule):
            if current_node not in graph:
                continue
            for neighbor in graph.neighbors(current_node):
                rel = graph[current_node][neighbor]['relation']
                if rel != target_rule[len(current_path)] or len(current_path) > len(target_rule):
                    continue
                queue.append((neighbor, current_path + [(current_node, rel,neighbor)]))
            for neighbor in graph.predecessors(current_node):
                rel = graph[neighbor][current_node]['relation']
                if rel != target_rule[len(current_path)] or len(current_path) > len(target_rule):
                    continue
                queue.append((neighbor, current_path + [(current_node, rel,neighbor)]))
    
    return result_paths
# relation_data_train = datasets.load_dataset('rmanluo/RoG-webqsp', split='train')
relation_data_train = datasets.load_dataset('rmanluo/RoG-cwq', split='validation')


# llm_chain = LLMChain(llm=model, prompt=multi_graph_groundness_prompt, verbose=True)
# batch_pred = llm_chain.batch([{"query": data[0]['RawQuestion'], "preceding_sentences": "", "evidence": " relationship: influence.influence_node.influenced_by",}], return_only_outputs=True)

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

In [9]:
id, sent_idx, q_entity, a_entity, inferential_chain, candidate relation, candidate entity

SyntaxError: invalid syntax (1468851539.py, line 1)

In [6]:
from typing import List
def save_to_json(data: List, data_path='../output/chain_data.json'):
    if not os.path.isfile(data_path):
        # 文件不存在，创建新列表并写入文件
        with open(data_path, 'w', encoding='utf-8') as file:
            json.dump(data, file, ensure_ascii=False, indent=4)
        return
    try:
        # 尝试读取现有文件
        with open(data_path, 'r', encoding='utf-8') as file:
            # 加载现有的JSON数据
            existing_data = json.load(file)
            existing_data.extend(data)
    except json.JSONDecodeError:
        # 文件不是有效的JSON，打印错误信息并退出
        print(f"文件 {data_path} 不是有效的JSON格式。")
        return
    except ValueError as e:
        # 打印错误信息并退出
        print(e)
        return
    # 将更新后的数据写回文件
    with open(data_path, 'w', encoding='utf-8') as file:
        json.dump(existing_data, file, ensure_ascii=False, indent=4)


In [7]:
with open('../output/chain_data_cwq_top_5.json', 'r') as f:
    now_data = json.load(f)

In [ ]:
chain_data = []
for index, line in enumerate(relation_data_train):

    id = line['id']
    topic_entity = line['q_entity']
    answer = line['a_entity']
    di_graph = build_Digraph(line['graph'])
    paths = get_truth_paths(topic_entity, answer, di_graph)
    # every question sample at most 3 paths
    paths = random.sample(paths, min(3, len(paths)))
    sent_idx = 0
    for pid,  p in enumerate(paths):
        preceding = ''
        for pidx, step in enumerate(p):
            candidate_relations = []
            real_relation = step[1]
            real_entity = step[2]
            candidate_entities =[tail for head, tail in di_graph.out_edges(step[0]) if di_graph[head][tail].get('relation') == step[1]]
            candidate_relation = [page.page_content.strip() for page in retriever.invoke(line['question'] + real_relation + real_entity)]
            if step[1] in candidate_relation:
                # recall precise
                chain_data.append({"qid": id,"sent_idx": sent_idx, "query": line['question'], "chain_step": pidx+1, "candidate_relation": candidate_relation, "candidate_entity": candidate_entities, "real_relation": real_relation, "real_entity": real_entity,"paths": p, "answer": answer, "effective": True})
            else:
            #     candidate_relation.append(step[1])
                chain_data.append({"qid": id,"sent_idx": sent_idx, "query": line['question'], "chain_step": pidx+1, "candidate_relation": candidate_relation, "candidate_entity": candidate_entities, "real_relation": real_relation, "real_entity": real_entity,"paths": p, "answer": answer, "effective": False})
            preceding += f' {step[1]}'
            sent_idx += 1
    if len(chain_data) > 100:
        print(f'Saving {index}')
        save_to_json(chain_data, f'../output/chain_data_top_5.json')
        chain_data = []



In [4]:
import json
with open('../output/chain_data_top_5.json', 'r', encoding='utf-8') as file:
    chain_data_web = json.load(file)
with open('../output/chain_data_cwq_top_5.json', 'r',encoding='utf-8') as file:
    chain_data_cwq =json.load(file)

#### create Retrieval data

In [ ]:
import random
input_data = []
# def construct_relevance(chain_data):
ids = []
for s, line in enumerate(random.sample(chain_data_cwq, 20000)):
    random.seed(s + 1)
    rand_num = random.random()
    if line['qid'] in ids:
        continue
    ids.append(line['qid'])
    if line['sent_idx'] == 0:
        if rand_num < 0.5:
            input_data.append({"input": {"query": line['query'], "preceding_sentences": "", "evidence": "", "target_output": ';'.join(line['answer'])}, "decision_token": "[New Retrieval]"})
            continue
    step = line['chain_step']
    if len(line['paths']) == 1:
        assert line['chain_step'] == 1
        if rand_num < 0.6:
            input_data.append({"input": {"query": line['query'], "preceding_sentences": "", "evidence": f"Relations retrieved: {line['real_relation']}\n Entities retrieved: {line['real_entity']}", "target_output": ';'.join(line['answer'])}, "decision_token": "[No Retrieval]"})
    else:
        if step == 1:
            if rand_num < 0.9:
                input_data.append({"input": {"query": line['query'], "preceding_sentences": "", "evidence": f"Relations retrieved: {line['real_relation']}\n Entities retrieved: {line['real_entity']}", "target_output": ';'.join(line['answer'])}, "decision_token": "[Continue to Retrieve Evidence]"})
        elif step < len(line['paths']):
            
            if rand_num < 0.8:
                input_data.append({"input": {"query": line['query'], "preceding_sentences": f"Information retrieved: {line['paths'][:step]}", "evidence": f"Relations retrieved: {line['real_relation']}\n Entities retrieved: {line['real_entity']}", "target_output": ';'.join(line['answer'])}, "decision_token": "[Continue to Retrieve Evidence]"})
        else:
            if rand_num < 0.4:
                input_data.append({"input": {"query": line['query'], "preceding_sentences": f"Information retrieved: {line['paths'][:step]}", "evidence": f"Relations retrieved: {line['real_relation']}\n Entities retrieved: {line['real_entity']}", "target_output": ';'.join(line['answer'])}, "decision_token": "[No Retrieval]"})


In [22]:
import random
# input_data = []

for s, line in enumerate(chain_data_web):
    random.seed(s + 1)
    rand_num = random.random()
    if line['sent_idx'] == 0:
        if rand_num < 0.5:
            input_data.append({"input": {"query": line['query'], "preceding_sentences": "", "evidence": "", "target_output": ';'.join(line['answer'])}, "decision_token": "[New Retrieval]"})
            continue
    step = line['chain_step']
    if len(line['paths']) == 1:
        assert line['chain_step'] == 1
        if rand_num < 0.6:
            input_data.append({"input": {"query": line['query'], "preceding_sentences": "", "evidence": f"Relations retrieved: {line['real_relation']}\n Entities retrieved: {line['real_entity']}", "target_output": ';'.join(line['answer'])}, "decision_token": "[No Retrieval]"})
    else:
        if step == 1:
            if rand_num < 0.9:
                input_data.append({"input": {"query": line['query'], "preceding_sentences": "", "evidence": f"Relations retrieved: {line['real_relation']}\n Entities retrieved: {line['real_entity']}", "target_output": ';'.join(line['answer'])}, "decision_token": "[Continue to Retrieve Evidence]"})
        elif step < len(line['paths']):
            if rand_num < 0.8:
                input_data.append({"input": {"query": line['query'], "preceding_sentences": f"Information retrieved: {line['paths'][:step]}", "evidence": f"Relations retrieved: {line['real_relation']}\n Entities retrieved: {line['real_entity']}", "target_output": ';'.join(line['answer'])}, "decision_token": "[Continue to Retrieve Evidence]"})
        else:
            if rand_num < 0.4:
                input_data.append({"input": {"query": line['query'], "preceding_sentences": f"Information retrieved: {line['paths'][:step]}", "evidence": f"Relations retrieved: {line['real_relation']}\n Entities retrieved: {line['real_entity']}", "target_output": ';'.join(line['answer'])}, "decision_token": "[No Retrieval]"})


In [23]:
retrival_critic = create_retrieval_data(input_data=input_data)

creating multi sentence retrieval data
{'instruction': 'You will be provided with a query, evidence, output sentence, and preceding sentences (optional). Your task is to determine whether the information in the output sentence can be fully verified by the evidence or if it requires further external verification. There are three cases:\n- If the output sentence can be verified solely with the evidence and the preceding sentences, then respond with [No Retrieval]. \n- If additional information about the tail entity in evidence is needed to verify the output sentence, respond with [Continue to Retrieve Evidence] \n- If more information unrelated to the evidence is needed, e.g. totally new relationship or new entity, reponse with [New Retrieval].\n\n', 'input': 'Query: what kind government does the us have\nEvidence: Relations retrieved: location.country.form_of_government\n Entities retrieved: Federal republic\nOutput: Presidential system;Federal republic;Constitutional republic', 'output

In [24]:
with open('../training_data/relevance_critic.json', 'w')as f:
    json.dump(retrival_critic, f)

#### create Support groundness data

In [6]:
# relationship relevance
graph_intepretable =  """You will receive a query, evidence and optional preceding sentences containing history information. The evidence contains graph relationships or entities may be useful to answer the query. Your task is to filters out valid information from the evidence to answer the given query, evaluate your output and provide explanations on your result.

###
Query: Name the president of the country whose main spoken language was Brahui in 1980?
Topic Entity: Brahui Language
Evidence: language.human_language.main_country; language.human_language.language_family; language.human_language.iso_639_3_code; base.rosetta.languoid.parent; language.human_language.writing_system; base.rosetta.languoid.languoid_class; language.human_language.countries_spoken_in; kg.object_profile.prominent_type; base.rosetta.languoid.document; base.ontologies.ontology_instance.equivalent_instances; base.rosetta.languoid.local_name; language.human_language.region
Preceding sentences: 
Output: 
1. {{language.human_language.main_country (Score: Fully relavant))}}: This relation is highly relevant as it directly relates to the country whose president is being asked for, and the main country where Brahui language is spoken in 1980.
2. {{language.human_language.countries_spoken_in (Score: Fully relavant)}}: This relation is also relevant as it provides information on the countries where Brahui language is spoken, which could help narrow down the search for the president.
3. {{base.rosetta.languoid.parent (Score: Partially relevant)}}: This relation is less relevant but still provides some context on the language family to which Brahui belongs, which could be useful in understanding the linguistic and cultural background of the country in question.

###
Query: {query}
Topic Entity: {topic}
Evidence: {evidence}
Preceding sentences: {preceding_sentences}
Output: 
"""
# The name of Justin Bieber's brother is Jaxon Bieber. This is based on the reasoning path that connects Justin Bieber to Jaxon Bieber through the relationship of sibling. The other paths that connect Justin Bieber to Jazmyn Bieber or back to Justin Bieber himself are incorrect in this context.
few_shot_intepretable_prompt = FewShotPromptTemplate(
        examples=[{
            "query": "Name the president of the country whose main spoken language was Brahui in 1980?",
            "topic": "Brahui Language",
            "evidence": "language.human_language.main_country; language.human_language.language_family; language.human_language.iso_639_3_code; base.rosetta.languoid.parent; language.human_language.writing_system; base.rosetta.languoid.languoid_class; language.human_language.countries_spoken_in; kg.object_profile.prominent_type; base.rosetta.languoid.document; base.ontologies.ontology_instance.equivalent_instances; base.rosetta.languoid.local_name; language.human_language.region",
            "preceding_sentences": "",
            "output": """1. {{language.human_language.main_country (Score: [Fully Relavant])}}: This relation is highly relevant as it directly relates to the country whose president is being asked for, and the main country where Brahui language is spoken in 1980.
2. {{language.human_language.countries_spoken_in (Score: [Fully Relavant])}}: This relation is also relevant as it provides information on the countries where Brahui language is spoken, which could help narrow down the search for the president.
3. {{base.rosetta.languoid.parent (Score: [Partially Relevant])}}: This relation is less relevant but still provides some context on the language family to which Brahui belongs, which could be useful in understanding the linguistic and cultural background of the country in question."""
        }],
        example_prompt=PromptTemplate.from_template("""###
Query: {query}
Topic Entity: {topic}
Evidence: {evidence}
Preceding sentences: {preceding_sentences}
Output: {output}
"""),
        prefix=
        """You will receive a query, topic entity, evidence and optional preceding sentences containing history information. The evidence contains graph relationships or entities may be useful to answer the query. Your task is to filters out 3 valid information from the evidence that contribute to answering the query and provide a relevance score for each output, output your explanations for the score.
The score of relevance range from [Fully Relavant], [Partially Relevant] to [Unrelevant].""",
        suffix=
        """###
Query: {query}
Topic Entity: {topic}
Evidence: {evidence}
Preceding sentences: {preceding_sentences}
Output: """,
        input_variables=["query", "evidence", "preceding_sentences", "topic"],
)
graph_intepretable_prompt = PromptTemplate(input_variables=["query", "evidence", "preceding_sentences", "topic"], template=
graph_intepretable)
llm_chain = LLMChain(llm=model, prompt=few_shot_intepretable_prompt, verbose=True)


In [7]:
llm_chain.batch([{"query": "when is the last time the the team has a team moscot named Lou Seal won the world series", "evidence": "baseball.baseball_league.teams;baseball.lifetime_batting_statistics.last_statistics_season;sports.sports_league.teams;sports.sports_team.team_mascot;baseball.baseball_team_stats.season", "preceding_sentences": '', 'topic': "Lou Seal"}])



> Entering new LLMChain chain...
Prompt after formatting:
You will receive a query, topic entity, evidence and optional preceding sentences containing history information. The evidence contains graph relationships or entities may be useful to answer the query. Your task is to filters out 3 valid information from the evidence that contribute to answering the query and provide a relevance score for each output, output your explanations for the score.
The score of relevance range from [Fully Relavant], [Partially Relevant] to [Unrelevant].

###
Query: Name the president of the country whose main spoken language was Brahui in 1980?
Topic Entity: Brahui Language
Evidence: language.human_language.main_country; language.human_language.language_family; language.human_language.iso_639_3_code; base.rosetta.languoid.parent; language.human_language.writing_system; base.rosetta.languoid.languoid_class; language.human_language.countries_spoken_in; kg.object_profile.prominent_type; base.rosetta.lan

[{'query': 'when is the last time the the team has a team moscot named Lou Seal won the world series',
  'evidence': 'baseball.baseball_league.teams;baseball.lifetime_batting_statistics.last_statistics_season;sports.sports_league.teams;sports.sports_team.team_mascot;baseball.baseball_team_stats.season',
  'preceding_sentences': '',
  'topic': 'Lou Seal',
  'text': '1. {sports.sports_team.team_mascot (Score: [Partially Relevant])}: This relation provides information on the team mascot, in this case Lou Seal, which is directly related to the query about the team with that mascot winning the world series. However, it does not specify when the team last won.\n2. {baseball.baseball_league.teams (Score: [Unrelevant])}: While this relation provides information on baseball teams, it does not specify any details about Lou Seal or when a team with that mascot last won the world series. \n3. {baseball.baseball_team_stats.season (Score: [Partially Relevant])}: This relation provides information on

In [ ]:
import re
import random
def extract_relationship_and_score(s):
    match_dict = dict()
    pattern = r'{(.+?) \(Score: (.+?)\)}'
    for match in re.findall(pattern, s):
        match_dict[match[0]] = match[1]
    return match_dict



In [74]:
input_data = []
batch_input = []
random.seed(42)
from openai import BadRequestError
qids = []
for line in random.sample(chain_data, 5000):
    if line['qid'] in qids:
        continue
    qids.append(line['qid'])
    batch_input.append({"query": line['query'], "evidence": ';'.join(line['candidate_relation']), "preceding_sentences": '', 'topic': line['paths'][line['chain_step'] - 1][0]})
    # try:
    #     batch_pred = llm_chain.batch([{"query": line['query'], "evidence": ';'.join(line['candidate_relation']), "preceding_sentences": '', 'topic': line['paths'][line['chain_step'] - 1][0]}], return_only_outputs=True)
    # except BadRequestError as e:
    #     print('*************************Bad Request**************')
    # except ValueError as e:
    #     print(f'******************Value Error****************************')
    # match_dict = extract_relationship_and_score(batch_pred[0]['text'])
    # input_data.append({"input": {"query": line['query'], "evidence": ';'.join(line['candidate_relation']), "preceding_sentences": '', 'topic': line['paths'][line['chain_step'] - 1][0]}, "match":match_dict, 'real_relation': line['real_relation']})
    # if len(input_data) >= 3:
    #     print('Saving')
    #     save_to_json(input_data, f'../output/relevance_cwq_critic_data_top_5_5000.json')
    #     input_data = []
# create_relevance_data(input_data)

In [ ]:
batch_size = 8
input_data = []
for num_batch in range(1, len(batch_input) // batch_size):
    start_index = (num_batch-1) * batch_size
    try:
        batch_pred = llm_chain.batch(batch_input[start_index: start_index + batch_size], return_only_outputs=True, max_concurrency=8)
    except BadRequestError as e:
        print('*************************Bad Request**************')
    except ValueError as e:
        print(f'******************Value Error****************************')
    match_dict = [extract_relationship_and_score(batch_pred[i]['text']) for i in range(len(batch_pred))]
    assert len(batch_pred) == batch_size
    for i in range(batch_size):
        line = batch_input[start_index + i]
        input_data.append({"input": line, "match":match_dict[i]})
    print(f'Saving batch{num_batch}')
    save_to_json(input_data, f'../output/relevance_cwq_critic_data_top_5_5000.json')
    input_data = []



In [ ]:
with open('../output/relevance_critic_data_top_5_5000.json', 'r') as f:
    data_1 = json.load(f)
with open('../output/relevance_cwq_critic_data_top_5_5000.json', 'r') as f:
    data_2 = json.load(f) 


In [ ]:
relevance_data = create_relevance_data(data_1 + data_2)

creating relevance data
{'instruction': 'You will receive a query, topic entity, evidence and optional preceding sentences containing history information. The evidence contains graph relationships or entities may be useful to answer the query. Your task is to filters out 3 valid information from the evidence that contribute to answering the query and provide a relevance score for each output.\nThe score of relevance range from [Fully relavant], [Partially relevant] to [Unrelevant].', 'input': 'Query: Which state has the official symbol of Charter Oak and is where Glastonbury is located?\nTopic entity: Glastonbury\nEvidence: government.governmental_jurisdiction.official_symbols;location.us_state.capital;location.country.national_anthem;location.country.form_of_government;location.location_symbol_relationship.symbolPreceding sentences: ', 'output': 'government.governmental_jurisdiction.official_symbols[Fully Relevant]; location.us_state.capital[Fully Relevant]; location.location_symbol_r

In [55]:
with open('../output/relevance_critic_data_top_5_5000.json', 'r') as f:
    data = json.load(f)
    print(len(data))

2200


#### create triplet relevance

In [65]:
# The name of Justin Bieber's brother is Jaxon Bieber. This is based on the reasoning path that connects Justin Bieber to Jaxon Bieber through the relationship of sibling. The other paths that connect Justin Bieber to Jazmyn Bieber or back to Justin Bieber himself are incorrect in this context.
few_shot_entity_prompt = FewShotPromptTemplate(
        examples=[{
            "query": "Name the president of the country whose main spoken language was Brahui in 1980",
            "evidence": "(Unknown-Entity, people.place_lived.location, De Smet)",
            "preceding_sentences": "(The Long Winter, book.written_work.author, Laura Ingalls Wilder);(Laura Ingalls Wilder, people.person.places_lived, Unknown-Entity);",
            "output": """1. {{De Smet (Score: [Fully Relavant])}}. The De Smet is fully relevant to the query as the triplet directly provide the answer to the query."""
        },
        {
            "query": "what is the name of justin bieber brother",
            "evidence": "(Justin Bieber, people.person.parents, Jeremy Bieber);(Justin Bieber, people.person.parents, Pattie Mallette)",
            "preceding_sentences": "",
            "output": """1. {{Jeremy Bieber (Score: [Partially Relavant])}}. Jeremy Bieber is rated as partially relevant because he is a potential candidate as Justin Bieber's brother based on the evidence provided.
2. {{Pattie Mallette (Score: [Unrelavant])}}. Pattie Mallette is rated as unrelevant as she is not the brother of Justin Bieber."""
        },
        {
            "query": "who did draco malloy end up marrying",
            "evidence": "(m.09k254w, fictional_universe.marriage_of_fictional_characters.spouses, Astoria Greengrass);(m.09k254w, fictional_universe.marriage_of_fictional_characters.spouses, Draco Malfoy)",
            "preceding_sentences": "(Draco Malfoy', 'fictional_universe.fictional_character.married_to', 'm.09k254w)",
            "output": """1. {{Astoria Greengrass (Score: [Fully Relavant])}}. Astoria Greengrass is fully relevant as she is identified as the character who Draco Malfoy ended up marrying based on the evidence provided.
2. {{Draco Malfoy (Score: [Unrelavant])}}.Draco Malfoy is rated as partially relevant because while he is mentioned in the evidence, the focus of the query is on who he married, not himself."""
        }],
        example_prompt=PromptTemplate.from_template("""###
Query: {query}
Evidence: {evidence}
Preceding sentences: {preceding_sentences}
Output: {output}
"""),
        prefix=
        """You will receive a query, evidence and optional preceding sentences containing history information. The evidence contains the associated retrieved knowledge graph triplets (head_entity, relation, tail_entity). Your task is to rate relevance score for each tail_entity from the evidence and output your explanations for the score.
The score of relevance range from [Fully Relavant], [Partially Relevant] to [Unrelevant].""",
        suffix=
        """###
Query: {query}
Evidence: {evidence}
Preceding sentences: {preceding_sentences}
Output: """,
        input_variables=["query", "evidence", "preceding_sentences", ],
)

llm_chain = LLMChain(llm=model, prompt=few_shot_entity_prompt, verbose=False)


In [59]:
from tqdm.notebook import tqdm

In [46]:
with open(f'../output/relevance_entity_webqsp.json', 'w') as f:
    json.dump(f)

TypeError: dump() missing 1 required positional argument: 'fp'

In [45]:
for r in relevance_entity:
    r['query'] = evidence_match[r['qid'] + '-' + str(r['sent_idx'])]

In [39]:
evidence_match[relevance_entity[0]['qid'] + '-' + str(relevance_entity[0]['sent_idx'])]

['(Justin Bieber, people.person.parents, Pattie Mallette)',
 '(Justin Bieber, people.person.parents, Jeremy Bieber)']

In [44]:
evidence_match = dict({})
for line in chain_data_web:
    if line['real_entity'].startswith('m.'):
        continue
    # evidence = ['({head}, {relation}, {tail})'.format_map({"head": line['paths'][line['chain_step'] - 1][0], "relation": line['paths'][line['chain_step'] - 1][1], "tail": tail}) for tail in line['candidate_entity']]
    evidence_match[line['qid'] + '-' + str(line['sent_idx'])] = line['query']

In [ ]:
input_data = []
random.seed(42)
#TODO:修改match匹配形式,修改preceding_sentences, candidate_entity个数过多规则问题
from openai import BadRequestError
qids = []
for index, line in enumerate(chain_data):
    # if line['qid'] in qids:
    #     continue
    if line['real_entity'].startswith('m.'):
        continue
    qids.append(line['qid'])
    evidence = ['({head}, {relation}, {tail})'.format_map({"head": line['paths'][line['chain_step'] - 1][0], "relation": line['paths'][line['chain_step'] - 1][1], "tail": tail}) for tail in line['candidate_entity']]
    try:
        batch_pred = llm_chain.batch([{"query": line['query'], "evidence": ';'.join(evidence), "preceding_sentences": ''}], return_only_outputs=True)
    except BadRequestError as e:
        print('*************************Bad Request**************')
    except ValueError as e:
        print(f'******************Value Error****************************')
    match_dict = extract_relationship_and_score(batch_pred[0]['text'])
    input_data.append({"qid":  line['qid'], "sent_idx": line['sent_idx'] ,"match":match_dict, 'output': batch_pred[0]['text'], "evidence": ';'.join(evidence)})
    if len(input_data) >= 10:
        print(f'Saving { index}')
        save_to_json(input_data, f'../output/relevance_entity_webqsp.json')
        input_data = []
# create_relevance_data(input_data)

Saving 15
Saving 30
Saving 41
Saving 54
******************Value Error****************************
Saving 69
******************Value Error****************************
Saving 80
Saving 91
Saving 104
Saving 117
Saving 131
Saving 147
Saving 159
Saving 170
Saving 183
Saving 198
Saving 210
Saving 227
Saving 245
Saving 260
Saving 275
Saving 287
Saving 303
Saving 315
Saving 326
Saving 345
Saving 357
Saving 367
Saving 384
Saving 396
Saving 408
Saving 425
Saving 435
Saving 453
Saving 467
Saving 477
Saving 489
Saving 499
Saving 509
Saving 519
Saving 530
Saving 540
Saving 553
Saving 563
Saving 578
Saving 589
Saving 604
Saving 615
Saving 625
Saving 637
Saving 647
Saving 658
Saving 673
Saving 686
Saving 702
Saving 718
Saving 730
Saving 746
Saving 757
Saving 769
Saving 782
Saving 793
Saving 803
Saving 813
Saving 824
Saving 835
Saving 846
Saving 861
Saving 874
Saving 884
Saving 897
Saving 910
Saving 920
Saving 930
Saving 940
Saving 950
Saving 962
Saving 976
Saving 990
Saving 1000
Saving 1014
Saving 10

Exception ignored in: <function tqdm.__del__ at 0x7f5230167b50>
Traceback (most recent call last):
  File "/media/disk1/chatgpt/miniconda3/envs/raptor/lib/python3.10/site-packages/tqdm/std.py", line 1148, in __del__
    self.close()
  File "/media/disk1/chatgpt/miniconda3/envs/raptor/lib/python3.10/site-packages/tqdm/notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'
Exception ignored in: <function tqdm.__del__ at 0x7f5230167b50>
Traceback (most recent call last):
  File "/media/disk1/chatgpt/miniconda3/envs/raptor/lib/python3.10/site-packages/tqdm/std.py", line 1148, in __del__
    self.close()
  File "/media/disk1/chatgpt/miniconda3/envs/raptor/lib/python3.10/site-packages/tqdm/notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'


Saving 1922
Saving 1934
Saving 1947
Saving 1960
Saving 1974
Saving 1988
Saving 1998
Saving 2009
Saving 2022
Saving 2032
Saving 2045
Saving 2056
Saving 2071
Saving 2082
Saving 2092
Saving 2110
Saving 2123
Saving 2133
Saving 2144
Saving 2156
Saving 2173
Saving 2185
Saving 2197
Saving 2209
Saving 2223
Saving 2236
Saving 2247
Saving 2258
Saving 2273
Saving 2287
Saving 2300
Saving 2313
Saving 2326
Saving 2336
Saving 2349
Saving 2361
Saving 2372
Saving 2383
Saving 2400
Saving 2413
Saving 2423
Saving 2434
Saving 2445
Saving 2461
Saving 2472
Saving 2486
Saving 2498
Saving 2510
Saving 2521
Saving 2531
Saving 2542
Saving 2556
Saving 2568
Saving 2582
Saving 2597
Saving 2613
Saving 2625
Saving 2639
Saving 2652
Saving 2665
Saving 2679
Saving 2692
Saving 2705
Saving 2718
Saving 2731
Saving 2747
Saving 2760
Saving 2776
Saving 2786
Saving 2799
Saving 2816
Saving 2829
******************Value Error****************************
******************Value Error****************************
Saving 2843
Saving 2

In [33]:
with open(f'../output/relevance_entity_webqsp.json', 'r') as f:
    relevance_entity = json.load(f)
    print(len(relevance_entity))

6110


#### create confidence data

In [35]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.chains import LLMChain
few_shot_intepretable_prompt = FewShotPromptTemplate(
        examples=[{
            "query": "what is the name of justin bieber brother?",
            "output": "[Retreive New Relationship]<paragraph>people.sibling_relationship.sibling;fictional_universe.fictional_character.siblings;fictional_universe.sibling_relationship_of_fictional_characters.siblings;people.person.sibling_s;people.family.members;people.person.parents</paragraph>Retrieved relationship: people.person.parents[Fully Relevant][Retrieve Entity]<paragraph>(Justin Bieber, people.person.parents, Pattie Mallette);(Justin Bieber, people.person.parents, Jeremy Bieber);(Justin Bieber, people.person.parents, Jeremy Bieber)</paragraph>Retrieved triplet: (Justin Bieber, people.person.parents, Jeremy Bieber)[Fully Relevant][Continue to Retrieve Evidence]<paragraph>people.sibling_relationship.sibling;people.person.sibling_s;people.person.parents;fictional_universe.fictional_character.siblings;people.family.members;people.person.children</paragraph>Retrieved relationship: people.person.children[Fully Relevant][Retrieve Entity]<paragraph>(Jeremy Bieber, people.person.children, Jazmyn Bieber);(Jeremy Bieber, people.person.children, Justin Bieber);(Jeremy Bieber, people.person.children, Jaxon Bieber);(Jeremy Bieber, people.person.children, Jaxon Bieber)</paragraph>Retrieved triplet: (Jeremy Bieber, people.person.children, Jaxon Bieber)[Fully Relevant][No Retrieval] Answer: Jaxon Bieber",
            "explanation": "The output provides the name of justin bieber's brother. This is based on the reasoning path that connects Justin Bieber to Jaxon Bieber through the relationship of sibling. The other paths that connect Justin Bieber to Jazmyn Bieber or back to Justin Bieber himself are incorrect in this context.",
            "rating": "[Confidence:5]"
        }],
        example_prompt=PromptTemplate.from_template("""
Query: {query}\n\nOutput: {output}
Explanation: {explanation}
Rating: {rating}
"""),
        prefix="""Given a query and an output, rate whether the response and the thought process appears to be a helpful and informative answer to the query, from 1 (lowest) - 5 (highest). We call this confidence score.
[Confidence:5]: The response provides a complete and correct reasoning chain to the query, and the final answer is complete and logically correct.
[Confidence:4]: The response mostly fulfills the need in the query and provides correct answers, while there can be some minor improvements such as shorter reasoning chain or less repetition.
[Confidence:3]: The response is acceptable, but the answers may be not complete or needs minor improvement.
[Confidence:2]: The reasoning process still addresses the main request, but the answers are not correct or not relevant to the query.
[Confidence:1]: The reasoning is barely irrelevant or does not give an answer in the end.""",
        suffix=
        """
Query: {query}\n\nOutput: {output}\n""",
        input_variables=["query", "output"],
)
# confidence_prompt = PromptTemplate(input_variables=["query", "output"], template=
# graph_intepretable)
llm_chain = LLMChain(llm=model, prompt=few_shot_intepretable_prompt, verbose=True)


In [37]:
llm_chain.invoke({'query': 'what is the money called in peru',"output": '[Retreive New Relationship]<paragraph>base.coinsdaily.denomination.money_value;finance.currency.currency_code;measurement_unit.money_value.amount;location.country.currency_used;location.country.currency_formerly_used</paragraph>Retrieved relationship: location.country.currency_used[Fully Relevant][Retrieve Entity] Justin' })



> Entering new LLMChain chain...
Prompt after formatting:
Given a query and an output, rate whether the response and the thought process appears to be a helpful and informative answer to the query, from 1 (lowest) - 5 (highest). We call this confidence score.
[Confidence:5]: The response provides a complete and correct reasoning chain to the query, and the final answer is complete and logically correct.
[Confidence:4]: The response mostly fulfills the need in the query and provides correct answers, while there can be some minor improvements such as shorter reasoning chain or less repetition.
[Confidence:3]: The response is acceptable, but the answers may be not complete or needs minor improvement.
[Confidence:2]: The reasoning process still addresses the main request, but the answers are not correct or not relevant to the query.
[Confidence:1]: The reasoning is barely irrelevant or does not give an answer in the end.


Query: what is the name of justin bieber brother?

Output: [Retre

{'query': 'what is the money called in peru',
 'output': '[Retreive New Relationship]<paragraph>base.coinsdaily.denomination.money_value;finance.currency.currency_code;measurement_unit.money_value.amount;location.country.currency_used;location.country.currency_formerly_used</paragraph>Retrieved relationship: location.country.currency_used[Fully Relevant][Retrieve Entity] Justin',
 'text': 'Rating: [Confidence:2]\nExplanation: The response does not provide the name of the currency in Peru, which is the main request of the query. Therefore, the reasoning process is not relevant to the query.'}

In [ ]:
with open("../data/WebQSP.train.json", "r") as f:
    train_webqsp = json.load(f)
id2chain = dict()
for line in train_webqsp['Questions']:
    unique_chain = []
    for p in line['Parses']:
        if p['InferentialChain'] not in unique_chain:
            unique_chain.append(p['InferentialChain'])
    id2chain[line['QuestionId']] = unique_chain


    

In [ ]:
input_data = []
#添加constraint
line = relation_data_train[1]
topic_entity = line['q_entity']
answer = line['a_entity']
di_graph = build_Digraph(line['graph'])
for chain in id2chain[line['id']]:
    paths = bfs_with_rule(di_graph, topic_entity[0], chain)
    # for p in paths:
        # if p[-1][-1] == answer[0]:
    input_data.append({"query": line['question'], "path": paths})
batch_pred = llm_chain.batch(input_data, return_only_outputs=True)

                          

In [ ]:
input_data = []
output = []
#添加constraint
from tqdm.notebook import tqdm
from openai import BadRequestError
ind = 1
with tqdm(total=len(relation_data_train), desc=f"Processing",ncols=1500) as pbar:
    while ind < len(relation_data_train):
        line = relation_data_train[ind]
        topic_entity = line['q_entity']
        answer = line['a_entity']
        di_graph = build_Digraph(line['graph'])
        for chain in id2chain[line['id']]:
            if chain:
                paths = bfs_with_rule(di_graph, topic_entity[0], chain)
            # for p in paths:
                # if p[-1][-1] == answer[0]:
            input_data.append({"query": line['question'], "path": paths})
        if len(input_data) > 8:
            try:
                batch_pred = llm_chain.batch(input_data, return_only_outputs=True)
            except BadRequestError as e:
                print('*************************Bad Request**************')
            except ValueError as e:
                print(f'******************Value Error {ind}****************************')
            output.extend(batch_pred)
            input_data = []

        ind += 1
        pbar.update(1)
        
                          

In [ ]:
def create_utility_data(input_data, multi_retrieval=False):
    print("creating groundness data")
    processed_data = []
    for item in input_data:
        if multi_retrieval is True:
            if "sent_idx" not in item or item["sent_idx"] == 0 or len(item["preceding_sentences"]) == 0:
                processed_data.append({"instruction": PROMPT_DICT["ground_multi_instruction"], "input": PROMPT_DICT["ground_multi_input_wo_preceding"].format_map(input), "output": item["score"], "task": "groudness"})
            else:
                processed_data.append({"instruction": PROMPT_DICT["ground_multi_instruction"], "input": PROMPT_DICT["ground_multi_input"].format_map(input), "output": item["score"], "task": "groudness"})
        else: 
            processed_data.append({"instruction": PROMPT_DICT["ground_instruction"], "input": PROMPT_DICT["ground_input"].format_map(item['input']), "output": item["score"], "task": "groudness"})
    print(processed_data[-1])
    print("total data number: {}".format(len(processed_data)))
    print(Counter([item["output"] for item in processed_data ]))
    return processed_data
process_data = create_groundness_data(input_data, multi_retrieval=False)

In [48]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.chains import LLMChain
graph_intepretable =  """Based on the reasoning paths, please answer the given question and explain why.
Question: {query}\n
Reasoning Paths: {path}\n
"""
# The name of Justin Bieber's brother is Jaxon Bieber. This is based on the reasoning path that connects Justin Bieber to Jaxon Bieber through the relationship of sibling. The other paths that connect Justin Bieber to Jazmyn Bieber or back to Justin Bieber himself are incorrect in this context.
few_shot_intepretable_prompt = FewShotPromptTemplate(
        examples=[{
            "query": "what is the name of justin bieber brother?",
            "path": "[[('Justin Bieber', 'people.person.sibling_s', 'm.0gxnnwc'), ('m.0gxnnwc', 'people.sibling_relationship.sibling', 'Jazmyn Bieber')], [('Justin Bieber', 'people.person.sibling_s', 'm.0gxnnwc'), ('m.0gxnnwc', 'people.sibling_relationship.sibling', 'Justin Bieber')], [('Justin Bieber', 'people.person.sibling_s', 'm.0gxnnwp'), ('m.0gxnnwp', 'people.sibling_relationship.sibling', 'Jaxon Bieber')], [('Justin Bieber', 'people.person.sibling_s', 'm.0gxnnwp'), ('m.0gxnnwp', 'people.sibling_relationship.sibling', 'Justin Bieber')]]",
            "answer": "Jaxon Bieber",
            "explanation": "This is based on the reasoning path that connects Justin Bieber to Jaxon Bieber through the relationship of sibling. The other paths that connect Justin Bieber to Jazmyn Bieber or back to Justin Bieber himself are incorrect in this context.Therefore, the name of Justin Bieber's brother is Jaxon Bieber"
        }],
        example_prompt=PromptTemplate.from_template("""
Question: {query}\n
Reasoning Paths: {path}\n
Answer: {answer}
Explanation: {explanation}
"""),
        prefix=
        """Based on the reasoning paths, please answer the given question and explain why.""",
        suffix=
        """
Question: {query}\n
Reasoning Paths: {path}\n""",
        input_variables=["query", "path"],
)
graph_intepretable_prompt = PromptTemplate(input_variables=["query", "path"], template=
graph_intepretable)
llm_chain = LLMChain(llm=model, prompt=few_shot_intepretable_prompt, verbose=False)


/tmp/ipykernel_422534/2392507187.py:32: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=model, prompt=few_shot_intepretable_prompt, verbose=False)


In [ ]:
本质是reflection的能力，reflection即对于思维和检索的内容进行打分

In [ ]:
with open("../output/groudness_webqsp.json", "w") as outfile:
        json.dump(process_data, outfile)

In [ ]:
import json
import random
random.shuffle(new_data)
output_file_name = '../output/testtime1_1105'
train_data = new_data[500:]
dev_data = new_data[:500]

with open(output_file_name + "_train.json", "w") as outfile:
    json.dump(train_data, outfile)
with open(output_file_name + "_dev.json", "w") as outfile:
    json.dump(dev_data, outfile)

In [ ]:
from collections import Counter
import random

new_data = []
for d in data:
    if d['output'] == '[Fully supported]':
        if random.random() < 0.2:
            new_data.append(d)
    else:
        new_data.append(d)
Counter([item["output"] for item in new_data ])


In [ ]:
with open("../output/intepretable_answer.json", "r") as f:
    data= json.load(f)
input_data = []
ind = 1
count = 0
llm_chain = LLMChain(llm=model, prompt=multi_graph_utility_prompt, verbose=True)   
while ind < len(relation_data_train):
        line = relation_data_train[ind]
        topic_entity = line['q_entity']
        answer = line['a_entity']
        di_graph = build_Digraph(line['graph'])
        for chain in id2chain[line['id']]:
            if chain:
                paths = bfs_with_rule(di_graph, topic_entity[0], chain)
                input_data.append({"query": line['question'], "output": '.'.join(data[count]['text'].split('\nExplanation: '))})
                count += 1
        if len(input_data) > 8:
            try:
                batch_pred = llm_chain.batch(input_data, return_only_outputs=True)
            except BadRequestError as e:
                print('*************************Bad Request**************')
            except ValueError as e:
                print(f'******************Value Error {ind}****************************')
            input_data = []
            break
   

### Inference data creation

In [14]:
with open('../output/chain_data_top_5.json', 'r') as f:
    chain_data = json.load(f)

In [15]:
len(chain_data)

7867

In [ ]:
new_start = True
generate_data = []
for line in chain_data:
    if new_start:
        starter = '[Retreive New Relationship]'
    if line['effective'] == False:
        processed_relationship = ';'.join(line['candidate_relation']  + [line['real_relation']])
    else:
        processed_relationship = ';'.join(line['candidate_relation'])
    candidate_entities = line['candidate_entity'][:5] if line['real_entity'] not in line['candidate_entity'] else line['candidate_entity'][:4] + [line['real_entity']]
    precessed_triplet = [f"({line['paths'][line['chain_step']-1][0]}, {line['paths'][line['chain_step']-1][1]}, {entity})" for entity in candidate_entities]
    starter += "<paragraph>{}</paragraph>".format(processed_relationship)
    starter += 'Retrieved relationship: {}[Fully Relevant][Retrieve Entity]'.format(line['real_relation'])
    starter += "<paragraph>{}</paragraph>".format(';'.join(precessed_triplet))
    starter +=f"Retrieved triplet: ({line['paths'][line['chain_step']-1][0]}, {line['paths'][line['chain_step']-1][1]}, {line['paths'][line['chain_step']-1][-1]})[Fully Relevant]"
    if line['chain_step'] == len(line['paths']):
        starter += f"[No Retrieval] {';'.join(line['answer'])} [Confidence: 5]"
        generate_data.append({"instruction": line['query'], "output": starter})
        new_start = True
        # if len(generate_data) == 10:
        #     break
    else:
        starter += f"[Continue to Retrieve Evidence]"
        new_start = False


In [18]:
with open('../output/test_generate_data_2000.json', 'w') as f:
    json.dump(generate_data, f)

In [ ]:
[New Retrieval] <paragraph> 'people.sibling_relationship.sibling',
'people.person.sibling_s',
'people.person.parents'</paragraph> Retrieved relationship: people.person.sibling_s[Fully Relevant][Retrieve entity]<paragraph> 'Jazmyn Bieber', 'Justin Bieber', 'Jaxon Bieber'</paragraph> Retrieved triplet: ('Jazmyn Bieber', 'sibling_s', 'Justin Bieber')[Fully Relevant][No Retrieval] Jaxon Bieber [Confidence: 5]


In [ ]:
from openai import OpenAI
client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="token-abc123",
    # skip_special_tokens=False,
)

completion = client.chat.completions.create(
  model="/media/disk1/chatgpt/LLaMA-Factory/saves/llama3-8b/full/sft/", 
  messages=[
    {"role": "user", "content": "You will be provided with a query, evidence, output sentence, and preceding sentences (optional). Your task is to determine whether the information in the output sentence can be fully verified by the evidence or if it requires further external verification. There are three cases:\n- If the output sentence can be verified solely with the evidence and the preceding sentences, then respond with [No Retrieval]. \n- If additional information about the tail entity in evidence is needed to verify the output sentence, respond with [Continue to Retrieve Evidence] \n- If more information unrelated to the evidence is needed, e.g. totally new relationship or new entity, reponse with [New Retrieval].\n\nQuery: what city was michael jackson born in?\nEvidence: \nOutput: Gary"}
  ]
)

step = 0
prediction = ''
preceding_sentences = ''
evidence = ''
i = 0
query = train_webqsp['Questions'][i]['RawQuestion']
candidate_entity = train_webqsp['Questions'][i]['Parses'][0]['TopicEntityName']
while prediction != '[No Retrieval]' and step < 3:
    step += 1

    current_input = PROMPT_DICT['multi_retrieval_three_way_instruction'] + PROMPT_DICT['multi_retrieval_three_way_input'].format(query=query, evidence=evidence, output=prediction, preceding_sentences=preceding_sentences)
    print(current_input)
    completion = client.chat.completions.create(
      model="/media/disk1/chatgpt/LLaMA-Factory/saves/llama3-8b/full/sft/", 
    
      messages=[
        {"role": "user", "content": current_input}
      ]
    )   
    prediction = completion.choices[0].message.content
        
    if prediction == '[Continue to Retrieve Evidence]':
      completion = client.chat.completions.create(
        model="/media/disk1/chatgpt/LLaMA-Factory/saves/llama3-8b/full/sft/", 
      
        messages=[
          {"role": "user", "content": }
        ]
      )
    prediction = completion.choices[0].message.content
      
    

In [17]:
PROMPT_DICT.keys()

dict_keys(['ground_instruction', 'ground_input', 'ground_instruction_multi', 'relevance_instruction', 'relevance_input', 'ground_multi_input', 'ground_multi_input_wo_preceding', 'retrieval_instruction', 'retrieval_input', 'retrieval_multi_instruction', 'retrieval_multi_input', 'multi_retrieval_three_way_instruction', 'multi_retrieval_three_way_input', 'multi_retrieval_three_way_input_wo_preceding', 'utility_instruction', 'utility_input'])

In [18]:
PROMPT_DICT['multi_retrieval_three_way_input']

'Query: {query}\nPreceding sentences: {preceding_sentences}\nEvidence: {evidence}\nOutput: {target_output}'